In [82]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

In [83]:
Cust = pd.read_csv('lv2-2401.csv')

In [84]:
Cust.shape

(2000, 9)

# 1

In [85]:
df1 = Cust.copy()

## 1-1

In [86]:
def get_age_p(age):
    if age < 20:
        return '10s'
    elif age < 30:
        return '20s'
    elif age < 40:
        return '30s'
    elif age < 50:
        return '40s'
    elif age < 60:
        return '50s'
    else:
        return 'other'

In [87]:
df1['AGE_P'] = df1['AGE'].map(get_age_p)

## 1-2

In [88]:
Agg24 = df1.groupby('AGE_P').apply(lambda x: pd.Series({
    'AVG_SPEND': x['SPENDING'].mean(),
    'CUST_CNT': x['C_ID'].shape[0]
}))

## 1-3

In [89]:
Agg24[Agg24['AVG_SPEND'] == Agg24['AVG_SPEND'].max()]

,AVG_SPEND,CUST_CNT
AGE_P,,
10s,16.125789,380.0


# 2

In [90]:
df2 = Cust.copy()

# 2-1

In [91]:
from sklearn.preprocessing import StandardScaler

In [92]:
col = ['AGE', 'INCOME', 'WORK_EXP', 'FAMILY', 'SPENDING']

In [93]:
df2[col] = pd.DataFrame(StandardScaler().fit_transform(df2[col]), columns=col)

## 2-2, 2-3

In [94]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [95]:
df2 = df2.sort_values(by='C_ID')

In [96]:
result = []

for i in [3, 4]:
    model = KMeans(n_clusters=i, random_state=1234, init='random').fit(df2[col])
    pred = model.predict(df2[col])
    score = silhouette_score(df2[col], pred)
    result.append({'n_cluster': i, 'score': score})

In [97]:
pd.DataFrame(result)['score'].max().round(3)

0.158

# 3

In [98]:
df3 = Cust.copy()

## 3-1

In [99]:
df3 = df3[~(df3['AGE'] < 14)].dropna(subset=['JOB'])

## 3-2

In [100]:
df3 = pd.get_dummies(df3, columns=['GENDER', 'JOB'], drop_first=True)

## 3-3

In [101]:
TrainSet6 = df3[df3['C_ID'] % 3 != 0]
TestSet6 = df3[df3['C_ID'] % 3 == 0]

## 3-4

In [102]:
from sklearn.tree import DecisionTreeClassifier

In [103]:
col = ['AGE', 'INCOME', 'WORK_EXP', 'FAMILY',
       'GENDER_Male',
       'JOB_B', 'JOB_C', 'JOB_D', 'JOB_E',
       'JOB_F', 'JOB_G', 'JOB_H', 'JOB_I']
len(col)

13

In [104]:
model = DecisionTreeClassifier(max_depth=5, random_state=1234).fit(X=TrainSet6[col], y=TrainSet6['GRADE'])

## 3-5

In [105]:
from sklearn.metrics import accuracy_score

In [106]:
pred = model.predict(X=TestSet6[col])

In [107]:
round(accuracy_score(y_true=TestSet6['GRADE'], y_pred=pred) * 100, 2)

33.22